In [1]:
!nvidia-smi

Tue Dec 30 08:06:32 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   65C    P8             12W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [5]:
%%writefile merge_sort.cu

#include <iostream>           // Для работы с вводом/выводом (cout)
#include <vector>             // Для использования динамического массива vector
#include <cstdlib>            // Для функции rand()
#include <chrono>             // Для измерения времени выполнения
#include <cuda_runtime.h>     // Для работы с CUDA API

using namespace std;          // Чтобы не писать std:: перед каждым элементом

// Функция слияния на GPU
__device__ void mergeDevice(int* arr, int left, int mid, int right, int* temp) { // Слияние двух подмассивов на GPU
    int i = left;              // Индекс для левой половины
    int j = mid;               // Индекс для правой половины
    int k = left;              // Индекс для записи в temp

    while (i < mid && j < right) { // Пока есть элементы в обеих половинах
        if (arr[i] <= arr[j]) temp[k++] = arr[i++]; // Берём элемент из левой половины
        else temp[k++] = arr[j++];                 // Берём элемент из правой половины
    }

    while (i < mid) temp[k++] = arr[i++];         // Копируем оставшиеся элементы левой половины
    while (j < right) temp[k++] = arr[j++];       // Копируем оставшиеся элементы правой половины

    for (int t = left; t < right; t++) arr[t] = temp[t]; // Переносим результат обратно в arr
}

// Kernel слияния
__global__ void mergeKernel(int* arr, int size, int width, int* temp) { // Kernel GPU
    int tid = blockIdx.x * blockDim.x + threadIdx.x;   // Глобальный идентификатор потока
    int left = tid * 2 * width;                        // Левая граница подмассива
    int mid = min(left + width, size);                // Средняя граница
    int right = min(left + 2 * width, size);          // Правая граница

    if (left < size && mid < size) {                  // Проверяем корректность границ
        mergeDevice(arr, left, mid, right, temp);    // Вызываем слияние для данного потока
    }
}

// Main
int main() {
    const int SIZE = 10000;           // Размер массива
    vector<int> h_arr(SIZE);          // Массив на CPU
    vector<int> temp(SIZE);           // Временный массив для слияния

    for (int i = 0; i < SIZE; i++) h_arr[i] = rand() % 100000; // Заполняем случайными числами

    cout << "Исходный массив (первые 20 элементов): ";
    for (int i = 0; i < 20 && i < SIZE; i++) cout << h_arr[i] << " "; // Выводим первые 20 элементов
    cout << endl;

    int* d_arr;
    int* d_temp;
    cudaMalloc(&d_arr, SIZE * sizeof(int));      // Выделяем память на GPU для массива
    cudaMalloc(&d_temp, SIZE * sizeof(int));    // Выделяем память на GPU для временного массива

    cudaMemcpy(d_arr, h_arr.data(), SIZE * sizeof(int), cudaMemcpyHostToDevice); // Копируем массив на GPU

    int threadsPerBlock = 256;                    // Потоки в одном блоке

    auto startTime = chrono::high_resolution_clock::now(); // Начало измерения времени

    for (int width = 1; width < SIZE; width *= 2) {   // Итеративное увеличение размера подмассива
        int numBlocks = (SIZE + 2 * width - 1) / (2 * width); // Количество блоков
        mergeKernel<<<numBlocks, threadsPerBlock>>>(d_arr, SIZE, width, d_temp); // Запуск kernel
        cudaDeviceSynchronize();                           // Ожидание завершения kernel
    }

    auto endTime = chrono::high_resolution_clock::now();   // Конец измерения времени
    cudaMemcpy(h_arr.data(), d_arr, SIZE * sizeof(int), cudaMemcpyDeviceToHost); // Копируем результат на CPU

    chrono::duration<double> duration = endTime - startTime; // Вычисляем время выполнения

    cout << "Отсортированный массив (первые 20 элементов): ";
    for (int i = 0; i < 20 && i < SIZE; i++) cout << h_arr[i] << " "; // Вывод первых 20 элементов после сортировки
    cout << endl;

    cout << "Размер массива: " << SIZE << endl;
    cout << "Время GPU сортировки: " << duration.count() << " секунд" << endl;

    cudaFree(d_arr);   // Освобождаем память GPU
    cudaFree(d_temp);

    return 0;          // Завершение программы
}




Overwriting merge_sort.cu


In [6]:
# Компиляция и запуск в Colab
!nvcc merge_sort.cu -o merge_sort
!./merge_sort

Исходный массив (первые 20 элементов): 89383 30886 92777 36915 47793 38335 85386 60492 16649 41421 2362 90027 68690 20059 97763 13926 80540 83426 89172 55736 
Отсортированный массив (первые 20 элементов): 89383 30886 92777 36915 47793 38335 85386 60492 16649 41421 2362 90027 68690 20059 97763 13926 80540 83426 89172 55736 
Размер массива: 10000
Время GPU сортировки: 0.00771202 секунд
